In [ ]:
from selenium import webdriver
from webdriver_auto_update import check_driver
check_driver(r'C:\Users\Tahsin Sayed\Downloads\chromedriver_win32')
import pandas as pd  
pd.options.mode.chained_assignment = None
import string
import time

In [ ]:
folder_path = r'C:\Users\Tahsin Sayed\Desktop\Image_to_text\\'
file_path = r'C:\Users\Tahsin Sayed\Desktop\Image_to_text\data file_xlsx'
file_path = r'C:\Users\Tahsin Sayed\Desktop\Image_to_text\Platform B\company_list.csv'
# df = pd.read_excel(folder_path+ 'data file_.xlsx')
df = pd.read_csv(file_path)
df.columns

In [ ]:
for i in df.index:
    x = df['NAMEOFISSUER'][i]
    x = ''.join('' if c in string.punctuation else c for c in x)
    df['Company_name'][i] = x.rsplit(' ', 1)[0]

In [ ]:
options = webdriver.ChromeOptions()
options.add_argument("user-data-dir=C:\\Users\\Tahsin Sayed\\AppData\\Local\\Google\\Chrome\\User Data")
driver = webdriver.Chrome(chrome_options=options)

In [ ]:
driver.get('https://kingscrowd.com/companies/search/')

In [ ]:
df

In [15]:
input_placeholder = driver.find_element_by_id('ratings_searchbox')
for i in df.index:
    #Skip the code if progress is completed.
    if df['Progress'][i] == 'Done':
        continue


    input_placeholder.clear()
    input_placeholder.send_keys(df['Company_name'][i])
    time.sleep(2)
    table = pd.read_html(driver.page_source)[0]
    # if i>=20:
    #     break
    
    if table['company'][0] == 'No matching records found':
        # There is no data for this company.
        df['Progress'][i] = 'Data Not Found'
        continue
    
    #Check start date. 
    table['start date'] = pd.to_datetime(table['start date'])
    table['close date'] = pd.to_datetime(table['close date'])
    instances = table[table['start date'].dt.year == df['Year'][i]].shape[0]

    #If Multiple Instance, need to check that manually. 
    if instances > 1:
        df['Progress'][i] = 'Multiple Data Found.'
        continue
    #If no data, log profress as 'No Data in this year'. 
    if instances == 0:
        df['Progress'][i] = 'No Data in This Year.'
        continue

    if instances == 1:
        data_index = table[table['start date'].dt.year == df['Year'][i]].index[0]
        print(data_index)


        # Accumalute Data into main Data frame
        df["Status"][i] = table['status'][data_index]
        df["Platform"][i] = table['platform'][data_index]
        df["Valuation"][i] = table['Valuation / Cap'][data_index]
        df["Security"][i] = table['security type'][data_index]
        df["Daily Raise"][i] = table['average daily raise'][data_index]
        df["Amount Raised"][i] = table['amount raised'][data_index]
        df["Start Date"][i] = table['start date'][data_index]
        df["Close Date"][i] = table['close date'][data_index]
        df["Investor"][i] = table['investors'][data_index]
        

        #Extract the link_to_page. 
        all_tables = driver.find_elements_by_xpath('//table')
        link_to_page = all_tables[0].find_elements_by_xpath('//a[@class="company-page-link"]')[data_index].get_attribute('href')
        driver.execute_script('window.open("http://bings.com","_blank");')
        driver.switch_to.window(window_name=driver.window_handles[-1])
        driver.get(link_to_page)
        divs = [x.text for x in driver.find_elements_by_xpath('//div[@class="cell small-4"]')]
        for each in divs:
            if 'Min. Goal' in each:
                df['Min Goal'][i] = each.split('\n')[-1]
                # print(each.split('\n')[-1])
            elif 'Max. Goal' in each:
                df['Max Goal'][i] = each.split('\n')[-1]
                # print(each.split('\n')[-1])
            elif 'Price Per Share' in each:
                df['Price Per Share'][i] = each.split('\n')[-1]
                # print(each.split('\n')[-1])
            elif 'Min. Investment' in each:
                df['Min Investment'][i] = each.split('\n')[-1]
                # print(each.split('\n')[-1])
            
        driver.find_element_by_id('company_profile-label').click()
        industry = [each.text.split('\n')[-1] for each in driver.find_elements_by_xpath('//div[@class="cell small-4"]') if 'Industry' in each.text]
        df['Industry'][i] = industry[0]


        driver.close()
        driver.switch_to.window(window_name=driver.window_handles[0])
        df['Progress'][i] = 'Done'
        
        '''
        Here remains the code. 

        '''
        pass

0
0
1
1
0
0
0
0
0
0
0
0
1
1
1
2
0
1
1
1
0
1
1


In [17]:
# df.to_csv('data.csv')
# df.to_excel('data.xlsx')
df.to_csv(r'C:\Users\Tahsin Sayed\OneDrive - BUET\Output doc file\Platform B\kingscrowd_data_B.csv')
df.to_excel(r'C:\Users\Tahsin Sayed\OneDrive - BUET\Output doc file\Platform B\kingscrowd_data_B.xlsx')

In [ ]:
df.head()